# Creando tu propio DataSet desde Google Imagenes
*by: Francisco Ingham and Jeremy Howard. Inspired by [Adrian Rosebrock](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)*m

en este turotial veremos como facilmente crear un dataset de imagenes de  Google. 
**Nota**: Puedes repetir estos pasos para cualquier categoria que quieras clasificar 

In [ ]:
from fastai.vision import *

## Obtener una lista de URLs

### Busca y Navega

Ir a [Google Images](http://images.google.com) y busca las imagenes en las cuales estas interesado. se específico en la busqueda de Google, obtendras mejores resultados y menos trabajo manual tienes que hacer.

Selecciona todas las imagenes que quieras bajar, o hasta ver un boton que dice 'mostrar mas resultados'. Todas las imagenes seleccionadas estan disponibles para bajar. Para obtener mas, click en el boton, y continua seleccionando. El número máximo de imagenes que Google muestra es 700.

Es una buena idea poner cosas que quieras excluir, Por ejemplo: si estas interesado en  Eurasian wolf, "canis lupus lupus", seria una buena idea excluir otros variantes:

    "canis lupus lupus" -dog -arctos -familiaris -baileyi -occidentalis

tambien puedes limitar tus resultados para mostrar sol fotos haciendo click en Herramientas y seleccionando Imagenes.

### Bajar a un archivo

Ahora debes ejecutar un codigo Javascript en tu explorador el cual grabara todas las URLs de todas las imagenes que quieres en tu dataset.

presiona <kbd>Ctrl</kbd><kbd>Shift</kbd><kbd>J</kbd> en Windows/Linux y <kbd>Cmd</kbd><kbd>Opt</kbd><kbd>J</kbd> en Mac, y una pequeña consola javascript aparecera. aca es donde puedes pegar tus comandos Javascript.

Necesitas las urls de lada una de las imagenes. Antes de ejecutar el siguiente comando, podrias querer deshabilitar y bloquear alguna extenciones (uBlock, AdBlockPlus etc.) en Chrome. de otra manera el comando window.open() no funcionara . Ahora basta ejecutar el siguiente comando:

```javascript
urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));

### Crea un directorio y sube las URL

Elige un nombre apropiado para tus imagenes etiquetadas. Puedes ejecutar esto multiples veces y crear etiquetas.

Los comandos siguientes deben ser ejecutados para cada categoria.

In [ ]:
folder = 'grizzly'
file = 'urls_grizzly.csv'

In [ ]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [ ]:
path.ls()

Finalmente, Solo necesitas presionar  'Upload' en tu directorio de trabajo y seleccionar tu archivo, Click en 'Upload' para cada uno de los archivos.

![imagen](images/download_images/upload.png)

## Bajar las imagenes

Ahora bajaran las imagenes de sus respectivas urls.

fast.ai tiene una funcion que permite que tu hagas esto. Solo debes especificar las urls tambien como la ruta destino y esta buncion bajará y guardara todas las imagenes que puedan ser abiertas. Si ellas tienen algún problema de apeertura, ellas no serán grabadas.

Bajemos tus imagenes! Nota que puedes elegir un máximo de imagenes a bajar. En este caso no bajaremos todas las urls.

Debes ejecutar esta linea para cada categoría.

In [ ]:
classes = ['teddys','grizzly','black']

In [ ]:
download_images(path/file, dest, max_pics=200)

Ahora podemos remover las imagenes que no pueden ser abiertas:

In [ ]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

## Revisemos la data

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

## Entrenemos el modelo

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
# Si el grafico no esta mostrando resultados definamos start|end learning rate
# learn.lr_find(start_lr=1e-5, end_lr=1e-1)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(3e-5,3e-4))

In [ ]:
learn.save('stage-2')

## Interpretación

In [ ]:
learn.load('stage-2');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

## Limpieza

Los errores No se deben a mala calidad de nuestro modelo. Hay imagenes que no deberian estar.

Utilizando `ImageCleaner` widget desde `fastai.widgets` Podemos eliminar, Remover fotos ue no pertenecen a las categorias.

In [ ]:
from fastai.widgets import *

Primero necesitamos ubicar estas imagness. Utilicemos `.from_toplosses`. Entonces podemos indexar las "malas imagenes" a la función  `ImageCleaner`.

las imagenes no serán borradas directamente del disco, secreara un nuevo archivo csv  `cleaned.csv` de donde se creara un nuevo grupo de imagenes etiquetadas de forma correcta y seguir entrenando el modelo.

Para limpiar un set completo de imagenes, Necesitamos crear un nuevo set de datos Sin la división. 

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .transform(get_transforms(), size=224)
                   .databunch()
     )

In [ ]:
# Si la data ya esta limpia utilizando `from_toplosses`,
# Ejecuta esta celda en vez de la anterior y procede a la eliminación de duplicados.
# Todos los resultados del paso anteriroo pueden ser sobreescritos
# con ek nuevo `ImageCleaner`.

# db = (ImageList.from_csv(path, 'cleaned.csv', folder='.')
#                    .no_split()
#                    .label_from_df()
#                    .transform(get_transforms(), size=224)
#                    .databunch()
#      )

Creemos un nuevo modelo utilizando el nuevo databunch con todas las imagenes.

In [ ]:
learn_cln = cnn_learner(db, models.resnet34, metrics=error_rate)

learn_cln.load('stage-2');

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn_cln)

Asegurate que estes ejecutando esto en Jupyter Notebook, no Jupyter Lab. Accesible via [/tree](/tree), no [/lab](/lab). Ejecutar `ImageCleaner` en Jupyter Lab no esta soportado [not currently supported](https://github.com/fastai/fastai/issues/1539).

In [ ]:
# No ejecutar en google colab o otra instancia de jupyter lab.
# Si ejecutas en Jupyter Lab, necesitas reiniciar tu instancia
# Todas laas variables se perderan.
ImageCleaner(ds, idxs, path)

Si el codigo anterior no muestra GUI(Conteniendo imagenes y botones), revisar las configuraciones "ipywidgets". Trata la solución en el siguiente link.

Etiqueta fotos para borrar presionando 'Delete'. Luego pincha 'Next Batch' para borrar las etiquetadas y mantener el resto. ImageCleaner mostrara una nueva fila de imagenes. En este caso, El widget mostrara imagenes hasta que no hayan mas en top_losses.ImageCleaner(ds, idxs)

YPuedes encontrar duplicadas y eliminarlas! Para hacer esto, necesitas ejecutar .from_similars para obtener potenciales duplicados ids y ejecuta ImageCleaner con duplicates=True. la API funciona igual que con imagenes mal clasificadas: solo selecciona las que quieres eliminar.

Asegurate de crear el databunch y learn_cln desde el archivo cleaned.csv. de otra forma el archivo podría ser sobreesrito desde cero, perdiendo todos los resultados obtenidos.

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn_cln)

Recuerda recrear ImageDataBunch desde `cleaned.csv` Para incluir los cambios en la Data

## Poniendo el modelo en producción

Primero, Exportemos el contenido del objeto `Learner` en producción:

In [ ]:
efaults.device = torch.device('cpu')

In [ ]:
img = open_image(path/'black'/'00000004.jpg')
img

Ahora creemos nuestro `Learner` en un ambiente productivo, solo asegurar que el  `path` contiene el archivo 'export.pkl'.

In [ ]:
learn = load_learner(path)

In [ ]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class

hora podemos crear una ruta como esta ([thanks](https://github.com/simonw/cougar-or-not) to Simon Willison for the structure of this code):

```python
@app.route("/classify-url", methods=["GET"])
async def classify_url(request):
    bytes = await get_bytes(request.query_params["url"])
    img = open_image(BytesIO(bytes))
    _,_,losses = learner.predict(img)
    return JSONResponse({
        "predictions": sorted(
            zip(cat_learner.data.classes, map(float, losses)),
            key=lambda p: p[1],
            reverse=True
        )
    })
```

(Este ejemplo es para [Starlette](https://www.starlette.io/) web app toolkit.)

## Posibles Errores
- La mayoria de las veces las cosas se entrenaran bien con defaults
- no hay mucho que "Tunear"
- La mayoria es
  - Learning rate
  - Number of epochs

### Learning rate (LR) Demasiado alta

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(1, max_lr=0.5)

### Learning rate (LR) demasiado baja

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-5)

In [ ]:
learn.recorder.plot_losses()

Esto toma mucho tiempo, Esta tomando mucho tiempo en cada imagen, Puede ser Sobreentrenamiento.

### Pocos epochs

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate, pretrained=False)

In [ ]:
learn.fit_one_cycle(1)

### Muchos epochs

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.9, bs=32, 
        ds_tfms=get_transforms(do_flip=False, max_rotate=0, max_zoom=1, max_lighting=0, max_warp=0
                              ),size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate, ps=0, wd=0)
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(40, slice(1e-6,1e-4))